In [1]:
from Common import gtf_manipulation
from pathlib import Path
import sys

import matplotlib.pyplot as plt
import pandas as pd
import sys
import math
import numpy as np

## get intron exon gtf


In [ ]:
# function to extract exons and intron; 
# only gene with at least one intron are considered

def get_intron(exons, gene_name):

    if len(exons) <= 1:
        return {}

    intron = set()
    exons = sorted(exons, key = lambda x: x["start"])
    for i in range(len(exons) - 1 ):
        intron.add((exons[i]["chr"], exons[i]["end"]-1, exons[i+1]["start"]-1,  gene_name, ".", exons[i]["strand"]))
    return intron

def get_exons(exons, gene_name):

    if len(exons) <= 1:
        return {}

    intron = set()
    exons = sorted(exons, key = lambda x: x["start"])
    for i in range(len(exons)):
        intron.add((exons[i]["chr"], exons[i]["start"], exons[i]["end"],  gene_name, ".", exons[i]["strand"]))
    return intron

In [ ]:
def get_attr(string):
    dico = {}
    spt_ = string.split(";")
    
    for x in spt_:
        if x:
            try:
                dico[x.split()[0].strip()] =  x.split()[1].replace('"', "").strip()
            except:
                print(spt_)
                raise
    return dico


def gtf_to_dict(gtf_file, main_ = "gene_id"):
    print(main_)
    dico = {}
    with open(gtf_file) as f_in:
        """atm reads only genes"""
        for line in f_in:
            line = line.strip()
            if not line:
                continue
            if line.startswith("#"):
                continue
            spt = line.strip().split("\t")
            try:
                chr_ = spt[0]
                start = int(spt[3]) #- 1 # gtf are 1 based
                end = int(spt[4]) #-1 
                strand =  spt[6]
                attr = get_attr(spt[-1])
                """                 if main_ not in attr:
                                    print(attr)
                continue """
                gene_id = attr[main_]
                gene_symbol = attr.get("gene_symbol", "None")
                type_ = spt[2]
            except:
                print(attr)
                print(line)
                print(spt)
                raise
            
            if gene_id not in dico:
                dico[gene_id] = {
                    "chr": chr_,
                    "symbol" :  gene_symbol,
                    "strand" : strand,
                    "transcript": {}
                }
                
            if type_ == "gene":
    
                dico[gene_id]["start"] = start
                dico[gene_id]["end"] = end
            
            else:
                try:
                    transcript_id  = attr.get("transcript_id", "None")
                    transcript_symbol = attr.get("transcript_symbol", "None")
                    
                    if transcript_id not in dico[gene_id]["transcript"]:
                        dico[gene_id]["transcript"][transcript_id] = {
                        "transcript_symbol" : transcript_symbol,
                        "transcript_id" : transcript_id
                        }
                    
                    if type_ not in dico[gene_id]["transcript"][transcript_id]: 
                        dico[gene_id]["transcript"][transcript_id][type_] = []
                        
                    dico[gene_id]["transcript"][transcript_id][type_].append({
                        "chr": chr_,
                        "start": start, 
                        "end" : end,
                        "strand" : strand,
                    })
                except: 
                    continue

In [ ]:
dmau_gtfdict = gtf_manipulation.gtf_to_dict("Currated_Mau_genes_formatted.gtf")
dsim_gtfdict = gtf_manipulation.gtf_to_dict("Currated_Sim_genes_formatted.gtf")


In [ ]:
sets = set()

for k, v in dsim_gtfdict.items():
    if tr_dico := v.get("transcript"):
        
        for tr in tr_dico:
            sets = sets.union(get_exons(tr_dico[tr]["exon"], "{}_{}".format(k, tr)))
            
            
            
with open("<dsim_exon.bed>", "w") as fo:
    seen = set()
    for e in sets:
        if (e[0], e[1], e[2]) in seen:
            continue
        else:
            seen.add((e[0], e[1], e[2]))
        fo.write("{}\n".format("\t".join(list(map(str, e)))))
    

In [ ]:
sets = set()

for k, v in dmau_gtfdict.items():
    if tr_dico := v.get("transcript"):
        
        for tr in tr_dico:
            sets = sets.union(get_exons(tr_dico[tr]["exon"], "{}_{}".format(k, tr)))
            
            
            
with open("<dmau_exon.bed>", "w") as fo:
    seen = set()
    for e in sets:
        if (e[0], e[1], e[2]) in seen:
            continue
        else:
            seen.add((e[0], e[1], e[2]))
        fo.write("{}\n".format("\t".join(list(map(str, e)))))

In [ ]:
sets = set()

for k, v in dsim_gtfdict.items():
    if tr_dico := v.get("transcript"):
        
        for tr in tr_dico:
            sets = sets.union(get_intron(tr_dico[tr]["exon"], "{}_{}".format(k, tr)))
            
            
            
with open("<dsim_intron.bed>", "w") as fo:
    seen = set()
    for e in sets:
        if (e[0], e[1], e[2]) in seen:
            continue
        else:
            seen.add((e[0], e[1], e[2]))
        fo.write("{}\n".format("\t".join(list(map(str, e)))))

In [ ]:
sets = set()

for k, v in dmau_gtfdict.items():
    if tr_dico := v.get("transcript"):
        
        for tr in tr_dico:
            sets = sets.union(get_intron(tr_dico[tr]["exon"], "{}_{}".format(k, tr)))
            
            
            
with open("<dmau_intron.bed>", "w") as fo:
    seen = set()
    for e in sets:
        if (e[0], e[1], e[2]) in seen:
            continue
        else:
            seen.add((e[0], e[1], e[2]))
        fo.write("{}\n".format("\t".join(list(map(str, e)))))

## match bed to genome

In [ ]:
def match_bed_to_genome_file(bed_file, genome_file, out):
    
    dico_bed = {}
    with open(bed_file) as fi:
        for l in fi:
            spt = l.strip().split()
            chr_ = spt[0]
            if chr_ not in dico_bed:
                dico_bed[chr_] = []
            dico_bed[chr_].append(l.strip())
            
    
    with open(out, "w") as f_o, open(genome_file) as fi:
        for l in fi:
            spt = l.split()
           
            if spt[0] in dico_bed:
                v = dico_bed[spt[0]]
                if len(v) == 1:
                    f_o.write(v[0] + "\n")
                else:
                    f_o.write("\n".join(dico_bed[spt[0]]) + "\n")
    

In [ ]:
### you do have to replace with the actual path in your computer

In [ ]:
# run for the intron
# for Dmau
match_bed_to_genome_file("<sorted dmau intron bed file>",
                         "<genome file dmau>",
                         "<dmau_intron.bamsorted.bed>")
# for dsim
match_bed_to_genome_file("<sorted dsim intron bed file>",
                         "<genome file dsim>",
                         "<dsim_intron.bamsorted.bed>")

In [ ]:
# run for the exon
# for Dmau
match_bed_to_genome_file("<sorted dmau exon bed file>",
                         "<genome file dmau>",
                         "<dmau_exon.bamsorted.bed>")
# for dsim
match_bed_to_genome_file("<sorted dsim intron bed file>",
                         "<genome file dsim>",
                         "<dsim_exon.bamsorted.bed>")

## plot

### Exon

In [ ]:
# coverage file exon
fsim = "<coverage_dsimExon_dsim_SRR22548176>"
fmau = "<coverage_dmauExon_dmau_SRR22548176>"


df_sim = pd.read_csv(fsim, sep="\t", header=None)
df_mau = pd.read_csv(fmau, sep="\t", header=None)


### dmau

In [ ]:

fig =  plt.figure(figsize=(10, 4))

X = list(df_mau[~df_mau[3].str.contains("FBgn")][8].values)
Y = list(df_mau[~df_mau[3].str.contains("FBgn")][9].values)

Xp = list(df_mau[df_mau[3].str.contains("FBgn")][8].values)
Yp = list(df_mau[df_mau[3].str.contains("FBgn")][9].values)

ax_scat = fig.add_axes((0.0, 0, 0.60, 0.85))
ax_hist_X_au = fig.add_axes((0.61, 0, 0.18, 0.85), sharey=ax_scat, frameon=False)
ax_hist_Y = fig.add_axes((0.80, 0, 0.18, 0.85), sharey=ax_scat, frameon=False)


ax_hist_X_au.hist(Y, color="#8da0cb", orientation="horizontal", range=(0, 1), bins=10)
ax_hist_Y.hist(Yp, color="#fc8d62", orientation="horizontal", range=(0, 1), bins=10)


#ax_hist.set_xlim(ax_hist.get_xlim()[::-1])
ax_hist_X_au.set_yticks([])
ax_hist_Y.set_yticks([])
ax_scat.scatter(X, Y, marker=".", color="#8da0cb", alpha=0.5)
ax_scat.scatter(Xp, Yp, marker=".", color="#fc8d62", alpha=1, s=100)


ax_scat.set_yticks([0, 0.25, 0.5, 0.75, 1], labels=["0", "0.25", "0.5", "0.75", "1"])
#ax_scat.set_xticks([100_000, 400_000, 700_000], labels = ["100kb", "400kb", "700kb"])
ax_scat.set(xlabel="exon len", ylabel="ratio base covered", title="exon bp covered ratio VS exon length")
#ax[1].set(xlabel="ratio coverage", ylabel="intron len")
ax_hist_Y.tick_params(labelleft=False, left=False)
ax_hist_X_au.tick_params(labelleft=False, left=False)

ax_hist_Y.set(title = "large Y genes", xlabel = "exon number")
ax_hist_X_au.set(title = "X and Autosomes", xlabel = "exon number")


fig.suptitle('vs. D.mauritiana(Exon)')
plt.savefig("dmau_exon_.pdf", bbox_inches="tight")

plt.savefig("dmau_exon.png", bbox_inches="tight")
plt.show();


### D sim

In [ ]:
fig =  plt.figure(figsize=(10, 4))

X = list(df_sim[~df_sim[3].str.contains("FBgn")][8].values)
Y = list(df_sim[~df_sim[3].str.contains("FBgn")][9].values)

Xp = list(df_sim[df_sim[3].str.contains("FBgn")][8].values)
Yp = list(df_sim[df_sim[3].str.contains("FBgn")][9].values)

ax_scat = fig.add_axes((0.0, 0, 0.60, 0.85))
ax_hist_X_au = fig.add_axes((0.61, 0, 0.18, 0.85), sharey=ax_scat, frameon=False)
ax_hist_Y = fig.add_axes((0.80, 0, 0.18, 0.85), sharey=ax_scat, frameon=False)


ax_hist_X_au.hist(Y, color="#8da0cb", orientation="horizontal", range=(0, 1), bins=10)
ax_hist_Y.hist(Yp, color="#fc8d62", orientation="horizontal", range=(0, 1), bins=10)


#ax_hist.set_xlim(ax_hist.get_xlim()[::-1])
ax_hist_X_au.set_yticks([])
ax_hist_Y.set_yticks([])
ax_scat.scatter(X, Y, marker=".", color="#8da0cb", alpha=0.5)
ax_scat.scatter(Xp, Yp, marker=".", color="#fc8d62", alpha=1, s=100)


ax_scat.set_yticks([0, 0.25, 0.5, 0.75, 1], labels=["0", "0.25", "0.5", "0.75", "1"])
#ax_scat.set_xticks([100_000, 400_000, 700_000], labels = ["100kb", "400kb", "700kb"])
ax_scat.set(xlabel="exon len", ylabel="ratio base covered", title="exon bp covered ratio VS exon length")
#ax[1].set(xlabel="ratio coverage", ylabel="intron len")
ax_hist_Y.tick_params(labelleft=False, left=False)
ax_hist_X_au.tick_params(labelleft=False, left=False)

ax_hist_Y.set(title = "large Y genes", xlabel = "exon number")
ax_hist_X_au.set(title = "X and Autosomes", xlabel = "exon number")


fig.suptitle('vs. D.simulans(exon)')
plt.savefig("dsim_exon.pdf", bbox_inches="tight")

plt.savefig("dsim_exon.png", bbox_inches="tight")
plt.show()

### Intron

In [ ]:
# coverage file Intron
fsim = "<coverage_dsimIntron_dsim_SRR22548176>"
fmau = "<coverage_dmauIntron_dmau_SRR22548176>"


df_sim = pd.read_csv(fsim, sep="\t", header=None)
df_mau = pd.read_csv(fmau, sep="\t", header=None)

In [ ]:
fig =  plt.figure(figsize=(10, 4))

X = list(df_mau[~df_mau[3].str.contains("FBgn")][8].values)
Y = list(df_mau[~df_mau[3].str.contains("FBgn")][9].values)

Xp = list(df_mau[df_mau[3].str.contains("FBgn")][8].values)
Yp = list(df_mau[df_mau[3].str.contains("FBgn")][9].values)

ax_scat = fig.add_axes((0.0, 0, 0.60, 0.85))
ax_hist_X_au = fig.add_axes((0.61, 0, 0.18, 0.85), sharey=ax_scat, frameon=False)
ax_hist_Y = fig.add_axes((0.80, 0, 0.18, 0.85), sharey=ax_scat, frameon=False)


ax_hist_X_au.hist(Y, color="#8da0cb", orientation="horizontal", range=(0, 1), bins=15)
ax_hist_Y.hist(Yp, color="#fc8d62", orientation="horizontal", range=(0, 1), bins=15)


#ax_hist.set_xlim(ax_hist.get_xlim()[::-1])
ax_hist_X_au.set_yticks([])
ax_hist_Y.set_yticks([])
ax_scat.scatter(X, Y, marker=".", color="#8da0cb", alpha=0.5)
ax_scat.scatter(Xp, Yp, marker=".", color="#fc8d62", alpha=1, s=100)


ax_scat.set_yticks([0, 0.25, 0.5, 0.75, 1], labels=["0", "0.25", "0.5", "0.75", "1"])
ax_scat.set_xticks([100_000, 400_000, 700_000], labels = ["100kb", "400kb", "700kb"])
ax_scat.set(xlabel="intron len", ylabel="ratio base covered", title="intron bp covered ratio VS intron length")
#ax[1].set(xlabel="ratio coverage", ylabel="intron len")
ax_hist_Y.tick_params(labelleft=False, left=False)
ax_hist_X_au.tick_params(labelleft=False, left=False)

ax_hist_Y.set(title = "large Y genes", xlabel = "intron number")
ax_hist_X_au.set(title = "X and Autosomes", xlabel = "intron number")


fig.suptitle('vs. D.mauritiana')
plt.savefig("dmau_intron.pdf", bbox_inches="tight")

plt.savefig("dmau_intron.png", bbox_inches="tight")
plt.show();


In [ ]:
fig =  plt.figure(figsize=(10, 4))

X = list(df_sim[~df_sim[3].str.contains("FBgn")][8].values)
Y = list(df_sim[~df_sim[3].str.contains("FBgn")][9].values)

Xp = list(df_sim[df_sim[3].str.contains("FBgn")][8].values)
Yp = list(df_sim[df_sim[3].str.contains("FBgn")][9].values)

ax_scat = fig.add_axes((0.0, 0, 0.60, 0.85))
ax_hist_X_au = fig.add_axes((0.61, 0, 0.18, 0.85), sharey=ax_scat, frameon=False)
ax_hist_Y = fig.add_axes((0.80, 0, 0.18, 0.85), sharey=ax_scat, frameon=False)


ax_hist_X_au.hist(Y, color="#8da0cb", orientation="horizontal", range=(0, 1), bins=10)
ax_hist_Y.hist(Yp, color="#fc8d62", orientation="horizontal", range=(0, 1), bins=10)

#ax_hist.set_xlim(ax_hist.get_xlim()[::-1])
ax_hist_X_au.set_yticks([])
ax_hist_Y.set_yticks([])
ax_scat.scatter(X, Y, marker=".", color="#8da0cb", alpha=0.5)
ax_scat.scatter(Xp, Yp, marker=".", color="#fc8d62", alpha=1, s=100)


ax_scat.set_yticks([0, 0.25, 0.5, 0.75, 1], labels=["0", "0.25", "0.5", "0.75", "1"])
ax_scat.set_xticks([100_000, 400_000, 700_000], labels = ["100kb", "400kb", "700kb"])
ax_scat.set(xlabel="intron len", ylabel="ratio base covered", title="intron bp covered ratio VS intron length")
#ax[1].set(xlabel="ratio coverage", ylabel="intron len")
ax_hist_Y.tick_params(labelleft=False, left=False)
ax_hist_X_au.tick_params(labelleft=False, left=False)

ax_hist_Y.set(title = "large Y genes", xlabel = "intron number")
ax_hist_X_au.set(title = "X and Autosomes", xlabel = "intron number")


fig.suptitle('vs. D.simulans')
plt.savefig("dsim_intron.pdf", bbox_inches="tight")

plt.savefig("dsim_intron.png", bbox_inches="tight")
plt.show()